In [1]:
import pandas as pd
from multicall.call import Call
from multicall.fetch_multicall_across_blocks import fetch_save_and_return, CACHE_PATH
from multicall.constants import W3
from multicall.cache import create_db, fetch_all_data
import sqlite3


def identity_function(x):
    return x


uniswap_v3_usdc_weth_pool = "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640"
weth = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
usdc = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"

weth_bal = Call(
    weth,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "weth_bal",
    identity_function,
)

await weth_bal(W3, 19_000_000)

RuntimeError: Cannot run the event loop while another loop is running

In [ ]:
# conn = sqlite3.connect(CACHE_PATH)

# cursor = conn.cursor()
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
# tables = cursor.fetchall()
# conn.close()
# tables

In [ ]:
fetch_all_data()

DatabaseError: Execution failed on sql 'SELECT * FROM multicallCache': no such table: multicallCache